# Customer purchase prediction

# Dataset preparation

## combine orders , reviews, payments dataset with customer dataset and dropping unwanted columns


In [101]:
# !pip install seaborn

In [102]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [103]:
item=pd.read_csv("../data/olist_order_items_dataset.csv")
order=pd.read_csv("../data/olist_orders_dataset.csv")
product=pd.read_csv("../data/olist_products_dataset.csv")
customer=pd.read_csv("../data/olist_customers_dataset.csv")
review=pd.read_csv("../data/olist_order_reviews_dataset.csv") 
payment=pd.read_csv("../data/olist_order_payments_dataset.csv")

In [104]:
df1=pd.merge(customer.drop(columns=['customer_zip_code_prefix']),order[['customer_id','order_id','order_purchase_timestamp']],on='customer_id')

In [105]:
df2 = pd.merge(df1,review[['order_id','review_score']],on='order_id')

In [106]:
paid = payment[['order_id','payment_value']].groupby('order_id').sum().reset_index()

In [107]:
df3 = pd.merge(df2,paid,on='order_id')

In [108]:
## making purchase date in datetime format
df3['order_purchase_timestamp']=pd.to_datetime(df3['order_purchase_timestamp']).dt.date

In [109]:
df3['order_purchase_timestamp']

0        2017-05-16
1        2018-01-12
2        2018-05-19
3        2018-03-13
4        2018-07-29
            ...    
99218    2018-04-07
99219    2018-04-04
99220    2018-04-08
99221    2017-11-03
99222    2017-12-19
Name: order_purchase_timestamp, Length: 99223, dtype: object

## We are going to sperate out 180 days (last 6 months) from the maximum day of purchase by customers out of the dataset. 2018/4 to 2018/10.
## We are using that data to predict whether the customer made a purchase in that period. We are going to use data until 2018/4 to make that prediction

In [110]:
number_of_days_for_purchase=180
max_date_in_data= df3['order_purchase_timestamp'].max()

In [111]:
min_date_in_data= df3['order_purchase_timestamp'].min()

In [112]:
min_date_in_data

datetime.date(2016, 9, 4)

In [113]:
from datetime import datetime, timedelta 

In [114]:
max_date_in_data

datetime.date(2018, 10, 17)

In [115]:
data_split_date=max_date_in_data -timedelta(days=number_of_days_for_purchase)

In [116]:
data_split_date

datetime.date(2018, 4, 20)

In [117]:
df_full=df3[df3['order_purchase_timestamp']<=data_split_date]
df_last=df3[df3['order_purchase_timestamp']>data_split_date]

In [118]:
df_last_180 =pd.DataFrame({'customer_unique_id':df3['customer_unique_id'].values.tolist()})

In [119]:
df_last_180=df_last_180.merge(df_last.groupby(['customer_unique_id'])['payment_value'].sum().reset_index(),how='outer',on='customer_unique_id')

In [120]:
df_last_180.fillna(0,inplace=True)

In [121]:
df_last_180['purchased']=np.where(df_last_180['payment_value']>0, 1,0)

In [122]:
df_last_180.head()

,customer_unique_id,payment_value,purchased
0,0000366f3b9a7992bf8c76cfdf3221e2,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,27.19,1
2,0000f46a3911fa3c0805444483337064,0.00,0
3,0000f6ccb0745a6a4b88665a16c9f078,0.00,0
4,0004aac84e0df4da2b147fca70cf8255,0.00,0


## The customers who bought items in that 6 months were given 1 and others were given 0. these values will act as binary classification for our prediction model

# Feature Engineering

In [123]:
## total amount per customer
tot_Amount=df_full.groupby('customer_unique_id')['payment_value'].sum().reset_index().rename(columns={'payment_value':'total_amount'})

In [124]:
## average review given
avg_review=df_full.groupby('customer_unique_id')['review_score'].mean().reset_index().rename(columns={'review_score':'avg_review'})

In [125]:
df_full

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_purchase_timestamp,review_score,payment_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,franca,SP,00e7ee1b050b8499577073aeb2a297a1,2017-05-16,4,146.87
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,2018-01-12,5,335.48
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,2018-03-13,5,173.30
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,jaragua do sul,SC,5741ea1f91b5fbab2bd2dc653a5b5099,2017-09-14,5,282.21
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,sao paulo,SP,36e694cf4cbc2a4803200c35e84abdc4,2018-02-19,5,22.77
...,...,...,...,...,...,...,...,...
99218,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,sao paulo,SP,6760e20addcf0121e9d58f2f1ff14298,2018-04-07,4,88.78
99219,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,taboao da serra,SP,9ec0c8947d973db4f4e8dcf1fbfa8f1b,2018-04-04,5,129.06
99220,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,fortaleza,CE,fed4434add09a6f332ea398efd656a5c,2018-04-08,1,56.04
99221,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,canoas,RS,e31ec91cea1ecf97797787471f98a8c2,2017-11-03,5,711.07


In [126]:
## months between first purchase and today
min_max_date=df_full.groupby('customer_unique_id')['order_purchase_timestamp'].agg([min,max])
min_max_date['max']=pd.to_datetime(min_max_date['max'])
min_max_date['min']=pd.to_datetime(min_max_date['min'])
today = pd.Timestamp(datetime.today())
min_max_date['diff_first_today']=(today-min_max_date['min']).dt.days

/var/folders/hy/c34mqxh123lgnfptrbk38tjw0000gn/T/ipykernel_62022/2566686634.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  min_max_date=df_full.groupby('customer_unique_id')['order_purchase_timestamp'].agg([min,max])
/var/folders/hy/c34mqxh123lgnfptrbk38tjw0000gn/T/ipykernel_62022/2566686634.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  min_max_date=df_full.groupby('customer_unique_id')['order_purchase_timestamp'].agg([min,max])


In [127]:
## months from first to last purchase
min_max_date['diff_first_last']=(min_max_date['max']-min_max_date['min']).dt.days

In [128]:
min_max_date

,min,max,diff_first_today,diff_first_last
customer_unique_id,,,,
0000f46a3911fa3c0805444483337064,2017-03-10,2017-03-10,2873,0
0000f6ccb0745a6a4b88665a16c9f078,2017-10-12,2017-10-12,2657,0
0004aac84e0df4da2b147fca70cf8255,2017-11-14,2017-11-14,2624,0
0004bd2a26a76fe21f786e4fbd80607f,2018-04-05,2018-04-05,2482,0
00050ab1314c0e55a6ca13cf7181fecf,2018-04-20,2018-04-20,2467,0
...,...,...,...,...
fffcc512b7dfecaffd80f13614af1d16,2018-04-11,2018-04-11,2476,0
fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08,2017-06-08,2783,0
fffea47cd6d3cc0a88bd621562a9d061,2017-12-10,2017-12-10,2598,0


In [129]:
## recency of Sales 
max_date=df_full['order_purchase_timestamp'].max()

min_max_date['recency'] = (np.datetime64(max_date)-min_max_date['max'])/np.timedelta64(1, 'm')

In [130]:
## Frequency of Sales
frequency=df_full.groupby('customer_unique_id')['order_id'].count().reset_index().rename(columns={'order_id':'frequency'})

In [131]:
## joining all the engineered features
dataset=pd.merge(tot_Amount,avg_review,on='customer_unique_id')
dataset=pd.merge(dataset,min_max_date,on='customer_unique_id')
dataset=pd.merge(dataset,frequency,on='customer_unique_id')
dataset=pd.merge(dataset,df_full[['customer_unique_id','customer_city','customer_state']],on='customer_unique_id')
dataset.drop(['min','max'],axis=1,inplace=True)

In [132]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler

In [133]:
### label encoding city and state names
encoder=LabelEncoder()
dataset['customer_city']=encoder.fit_transform(dataset['customer_city'])
dataset['customer_state']=encoder.fit_transform(dataset['customer_state'])

In [134]:
##merging with the label dataset we have created 
dataset_full=dataset.merge(df_last_180[['customer_unique_id','purchased']],on='customer_unique_id')
dataset_full.drop(columns='customer_unique_id',inplace=True)

# Evaluating Machine learning models

### we are going to do binary classification to predict whether a customer will purchase within the next 6 months Since this is a classification problem we use several models here , For comparing the models we use metrics like r2 score and accuracy score.


1. Linear Regression
2. Random Forest Classifier
3. Extra Trees Classifier
4. Gradient Boost Classifier
5. K nearest nerighbour Classifier

In [135]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score,accuracy_score,classification_report
# Splitting data into training/testing
from sklearn.model_selection import train_test_split,GridSearchCV

In [136]:
##splitting to train and test dataset
X_train,X_test,y_train,y_test=train_test_split(dataset_full.iloc[:,:-1],dataset_full.iloc[:,-1], test_size=0.2, random_state=31)

In [137]:
## calculating gini scores for the models
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(1/n_samples, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)
    
    # normalize to true Gini coefficient
    return G_pred/G_true

In [138]:
# Evaluate several ml models by training on training set and testing on testing set
def evaluate(X_train, X_test, y_train, y_test):
    # Names of models
    model_name_list = ['Linear Regression',
                      'Random Forest', 'Extra Trees',
                       'Gradient Boosted','KNeighbors']

    
    # Instantiate the models
    model1 = LinearRegression()
    model3 = RandomForestClassifier(n_estimators=50)
    model4 = ExtraTreesClassifier(n_estimators=50)
    model6 = GradientBoostingClassifier(n_estimators=20)
    model7= KNeighborsClassifier(n_neighbors = 5)
    
    # Dataframe for results
    results = pd.DataFrame(columns=['r2', 'accuracy','gini'], index = model_name_list)
    
    # Train and predict with each model
    for i, model in enumerate([model1, model3, model4, model6,model7]):
   
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
               
        # Metrics
        r2 = r2_score(y_test,predictions)
        preds=np.where(predictions>0.5,1,0)
        accuracy = accuracy_score(y_test,preds)
        gini=Gini(y_test,preds)
        
        # Insert results into the dataframe
        model_name = model_name_list[i]
        results.loc[model_name, :] = [r2, accuracy,gini]
    
    return results

In [139]:
results=evaluate(X_train, X_test, y_train, y_test)

In [140]:
results

,r2,accuracy,gini
Linear Regression,0.047211,0.97692,0.00262
Random Forest,0.824282,0.996038,0.879507
Extra Trees,0.807547,0.995661,0.88838
Gradient Boosted,0.104676,0.979813,0.128741
KNeighbors,0.297129,0.984152,0.419302


### As r2 score reaches 1, the model is much capable of explaining the variance in purchase probability prediction of the customers. So Random Forest Classifier is the best classifier for the prediction.

### The gini coefficient of the forest classifier and Extratrees classifier is closer to one, meaning there is inequality in the predicted values